# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

LANGSMITH_API_KEY=****60e8
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials
AZURE_OPENAI_PROJECT_ENDPOINT=****ject
AZURE_OPENAI_API_KEY=****U7lG
AZURE_OPENAI_ENDPOINT=****com/
AZURE_OPENAI_API_VERSION=****view
AZURE_OPENAI_DEPLOYMENT=****mini


In [4]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [5]:
from langchain.agents import create_agent
from dotenv import load_dotenv
import os
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),  # or your deployment
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


agent_with_mcp = create_agent(
    model=llm,
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [7]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the time in Hyderabad, India right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the time in Hyderabad, India right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (call_CxhVjuHNhf774iJ7UpG0GMYq)
 Call ID: call_CxhVjuHNhf774iJ7UpG0GMYq
  Args:
    timezone: Asia/Kolkata
================================= Tool Message =================================
Name: current_time

2026-01-24T19:45:28+05:30
================================== Ai Message ==================================

The current time in Hyderabad, India is 7:45 PM on January 24, 2026.
